In [2]:
import cv2
import numpy as np

In [3]:
img_path = 'C:\\lee\\work\\sample\\Sample_id_3\\0.cam_rearview.f_1.rgb.png'
resave_path = 'C:\\lee\\work\\sample\\Sample_id_3'

In [26]:
def transform_rect_to_ellipse(image_path):
    # 직사각형 이미지 불러오기
    img = cv2.imread(image_path)

    # 이미지의 높이와 너비 구하기
    height, width = img.shape[:2]

    # 타원의 중심 좌표 계산
    center_x = width // 2
    center_y = height // 2

    # 타원의 장축과 단축 길이 설정 (예: 여기서는 직사각형의 반 너비와 반 높이)
    major_axis = width*3 // 5
    minor_axis = height*3 // 5

    # 타원의 회전 각도
    angle = 0

    # 타원 변환을 위한 변환 행렬 계산
    rotation_matrix = cv2.getRotationMatrix2D((center_x, center_y), angle, 1)
    transformed_img = cv2.warpAffine(img, rotation_matrix, (width, height))

    # 타원 영역을 나타내기 위한 마스크 생성
    mask = np.zeros_like(img)
    cv2.ellipse(mask, (center_x, center_y), (major_axis, minor_axis), angle, 0, 360, (255, 255, 255), -1)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    # 타원 영역 외의 부분을 제거
    result = cv2.bitwise_and(transformed_img, transformed_img, mask=mask)

    return result

In [27]:
transformed_image = transform_rect_to_ellipse(img_path)
cv2.imwrite(f"{resave_path}\\ellipse_image.jpg", transformed_image)

True

In [12]:
def fisheye_effect(image_path, strength=1, zoom=0.8):
    # 이미지 불러오기
    img = cv2.imread(image_path)
    height, width = img.shape[:2] #이 부분에서 Focal Length를 픽셀로 변환하여 그 값을 넣는 것이 더 일반적임 
    
    # 왜곡 파라미터 설정
    K = np.array([[width, 0, width / 2],
                  [0, height, height / 2],
                  [0, 0, 1]], dtype=np.float32)
    D = np.array([strength, 0, 0, 0], dtype=np.float32)
    
    # 왜곡 맵 생성
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, (width, height), cv2.CV_16SC2)
    
    # 왜곡 적용
    fisheye_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    
    # 결과 이미지 반환
    return pincushion_img, map1, map2


In [ ]:
def transform_landmarks(landmarks, map1, map2):
    new_landmarks = []
    for (x, y) in landmarks:
        new_x = map1[y, x, 0]
        new_y = map1[y, x, 1]
        new_landmarks.append((new_x, new_y))
    return new_landmarks

In [13]:
transformed_image, map1, map2 = fisheye_effect(img_path)

cv2.imwrite(f"{resave_path}\\fisheye_image.jpg", transformed_image)

True

In [ ]:
transformed_landmarks = transform_landmarks(landmarks, map1, map2)
print(transformed_landmarks)